In [23]:
# trd
import MetaTrader5 as mt

# processing
import pandas as pd
from datetime import datetime

---

In [24]:
mt.initialize()

True

In [25]:
credentials = open('../.pass', 'r').read().split('\n')
login = int(credentials[0])
password = credentials[1]
server = credentials[2]

mt.login(login, password, server)

True

In [26]:
mt.account_info()

AccountInfo(login=152702909, trade_mode=0, leverage=2, limit_orders=1024, margin_so_mode=0, trade_allowed=True, trade_expert=True, margin_mode=2, currency_digits=2, fifo_close=False, balance=584.23, credit=0.0, profit=0.0, equity=584.23, margin=0.0, margin_free=584.23, margin_level=0.0, margin_so_call=30.0, margin_so_so=0.0, margin_initial=0.0, margin_maintenance=0.0, assets=0.0, liabilities=0.0, commission_blocked=0.0, name='DEMO', server='Exness-MT5Trial', currency='USD', company='Exness Technologies Ltd')

In [33]:
df = pd.DataFrame(mt.copy_rates_range('BTCUSD', mt.TIMEFRAME_D1, datetime(2023, 1, 1), datetime.now()))
df.head()

,time,open,high,low,close,tick_volume,spread,real_volume
0,1672531200,16532.88,16619.48,16492.71,16603.49,79688,427,0
1,1672617600,16602.97,16773.48,16535.88,16665.66,82223,747,0
2,1672704000,16665.36,16773.61,16593.15,16662.49,82161,575,0
3,1672790400,16661.05,16988.17,16639.02,16837.79,83272,371,0
4,1672876800,16839.31,16868.59,16750.72,16814.80,82185,517,0


In [34]:
df.time = pd.to_datetime(df.time, unit='s', utc=True)
df.head()

,time,open,high,low,close,tick_volume,spread,real_volume
0,2023-01-01 00:00:00+00:00,16532.88,16619.48,16492.71,16603.49,79688,427,0
1,2023-01-02 00:00:00+00:00,16602.97,16773.48,16535.88,16665.66,82223,747,0
2,2023-01-03 00:00:00+00:00,16665.36,16773.61,16593.15,16662.49,82161,575,0
3,2023-01-04 00:00:00+00:00,16661.05,16988.17,16639.02,16837.79,83272,371,0
4,2023-01-05 00:00:00+00:00,16839.31,16868.59,16750.72,16814.80,82185,517,0


In [35]:
df.tail()

,time,open,high,low,close,tick_volume,spread,real_volume
424,2024-02-29 00:00:00+00:00,62457.02,63659.37,60308.47,61164.14,256881,1970,0
425,2024-03-01 00:00:00+00:00,61156.81,63185.63,60704.05,62363.47,194417,2047,0
426,2024-03-02 00:00:00+00:00,62379.96,62448.94,61562.16,61995.21,138529,2102,0
427,2024-03-03 00:00:00+00:00,61986.62,63266.51,61310.22,63165.73,144740,1829,0
428,2024-03-04 00:00:00+00:00,63145.13,64293.67,62354.76,63693.22,21919,1962,0


In [33]:
mt.orders_get()

()

In [16]:
mt.positions_total()

1

In [35]:
mt.positions_get()

(TradePosition(ticket=582939099, time=1708840045, time_msc=1708840045887, time_update=1708840045, time_update_msc=1708840045887, type=0, magic=0, identifier=582939099, reason=3, volume=0.01, price_open=51675.89, sl=0.0, tp=0.0, price_current=51638.53, swap=0.0, profit=-0.37, symbol='BTCUSD', comment='python script bu', external_id=''),)

In [38]:
def order_buy(symbol: str, volume: float, sl: float = None, tp: float = None):
    price = mt.symbol_info_tick(symbol).ask
    request = {
        "action": mt.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": volume,
        "type": mt.ORDER_TYPE_BUY,
        "price": price,
        "comment": "PyScriptBuy",
        "type_time": mt.ORDER_TIME_GTC,
        "type_filling": mt.ORDER_FILLING_IOC,
    }
    
    # set stoploss
    if sl: request.update({"sl": price * (1 - sl)})
    
    # set take-profit
    if tp: request.update({"tp": price * (1 + tp)})
    
    # execute order
    return mt.order_send(request)


def order_buy_close(position: mt.TradePosition):
    request = {
        "action": mt.TRADE_ACTION_DEAL,
        "symbol": position.symbol,
        "volume": position.volume,
        "type": mt.ORDER_TYPE_SELL,
        "position": position.ticket,
        "price": mt.symbol_info_tick(position.symbol).bid,
        "comment": "PyScriptBuyClose",
        "type_time": mt.ORDER_TIME_GTC,
        "type_filling": mt.ORDER_FILLING_IOC,
    }
    
    # execute order
    return mt.order_send(request)


In [42]:
def order_sell(symbol: str, volume: float, sl: float = None, tp: float = None):
    price = mt.symbol_info_tick(symbol).bid
    request = {
        "action": mt.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": volume,
        "type": mt.ORDER_TYPE_SELL,
        "price": price,
        "comment": "PyScriptSell",
        "type_time": mt.ORDER_TIME_GTC,
        "type_filling": mt.ORDER_FILLING_IOC,
    }
    
    # set stoploss
    if sl: request.update({"sl": price * (1 + sl)})
    
    # set take-profit
    if tp: request.update({"tp": price * (1 - tp)})
    
    # execute order
    return mt.order_send(request)


def order_sell_close(position: mt.TradePosition):
    request = {
        "action": mt.TRADE_ACTION_DEAL,
        "symbol": position.symbol,
        "volume": position.volume,
        "type": mt.ORDER_TYPE_BUY,
        "position": position.ticket,
        "price": mt.symbol_info_tick(position.symbol).ask,
        "comment": "PyScriptSellClose",
        "type_time": mt.ORDER_TIME_GTC,
        "type_filling": mt.ORDER_FILLING_IOC,
    }
    
    # execute order
    return mt.order_send(request)

In [40]:
symbol = 'BTCUSD'
volume = 0.01

In [20]:
result = order_buy('BTCUSD', 0.01)
result.comment

'PyScriptBuy'

In [43]:
result_close = order_buy_close(mt.positions_get()[0])
result_close.comment

'Market closed'

In [41]:
result = order_sell('BTCUSD', 0.01)
result.comment

'PyScriptSell'

In [23]:
# prepare the buy request structure
lot = 0.001
symbol = "BTCUSD"
request = {
    "action": mt.TRADE_ACTION_DEAL,
    "symbol": symbol,
    "volume": lot,
    "type": mt.ORDER_TYPE_BUY,
    "price": mt.symbol_info_tick(symbol).ask,
    "sl": None,
    "tp": None,
    "comment": "python script buy",
    "type_time": mt.ORDER_TIME_GTC,
    "type_filling": mt.ORDER_FILLING_IOC,
}

# execute order
result = mt.order_send(request)
result

In [46]:
result.order

575977263

In [47]:
# prepare the buy request structure
request = {
    "action": mt.TRADE_ACTION_DEAL,
    "symbol": symbol,
    "volume": lot,
    "type": mt.ORDER_TYPE_SELL,
    "position": result.order,
    "price": mt.symbol_info_tick(symbol).bid,
    "comment": "python script close",
    "type_time": mt.ORDER_TIME_GTC,
    "type_filling": mt.ORDER_FILLING_IOC,
}

# execute order
result = mt.order_send(request)
result

OrderSendResult(retcode=10009, deal=363851543, order=575977435, volume=0.01, price=51746.37, bid=51746.37, ask=51775.23, comment='python script cl', request_id=2878956849, retcode_external=0, request=TradeRequest(action=1, magic=0, order=0, symbol='BTCUSD', volume=0.01, price=51746.37, stoplimit=0.0, sl=0.0, tp=0.0, deviation=0, type=1, type_filling=1, type_time=0, expiration=0, comment='python script close', position=575977263, position_by=0))

In [48]:
mt.order_check(request)

OrderCheckResult(retcode=10013, balance=0.0, equity=0.0, profit=0.0, margin=0.0, margin_free=0.0, margin_level=0.0, comment='Invalid request', request=TradeRequest(action=1, magic=0, order=0, symbol='BTCUSD', volume=0.01, price=51746.37, stoplimit=0.0, sl=0.0, tp=0.0, deviation=0, type=1, type_filling=1, type_time=0, expiration=0, comment='python script close', position=575977263, position_by=0))

In [44]:
mt.shutdown()

True